In [17]:
import datetime
import pandas
import seaborn as sns
from davitpy import pydarn
from davitpy import gme
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib.dates import date2num, DateFormatter, MinuteLocator
from matplotlib.ticker import FuncFormatter, MaxNLocator
import numpy as np
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
# function to convert tec date, time string to python datetime obj
def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
    if row["timeStr"] < 10:
        currTimeStr = "000" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 100:
        currTimeStr = "00" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 1000:
        currTimeStr = "0" + str( int( row["timeStr"] ) )
    else:
        currTimeStr = str( int( row["timeStr"] ) )  
    #print datetime.datetime.strptime( currDateStr+ ":" + currTimeStr, "%Y%m%d:%H%M" )
    return datetime.datetime.strptime( currDateStr+ ":" + currTimeStr, "%Y%m%d:%H%M" )

In [19]:
#loading the TEC data into pandas dataframe
#initializing the columns in the table
inpColList = [ "dateStr", "timeStr", "Mlat", "Mlon", "med_tec", "dlat", "dlon" ]
inpFile = "/home/sneha/GSocProjectSpaceVT/dataset/med_filt_tec/tec-medFilt-20150101.txt"
medFiltTECDF = pandas.read_csv(inpFile, delim_whitespace=True, header=None, names=inpColList)
medFiltTECDF["date"] = medFiltTECDF.apply(convert_to_datetime, axis=1)

In [20]:
#selecting the longitudes in the north-american sector
longitudes = list(range(250, 362, 2)) + list(range(0, 35, 2))
print longitudes
#selecting only those rows whose longitudes is in the range of north-american sector
selectedmedFiltTECDF = medFiltTECDF[medFiltTECDF["Mlon"].isin(longitudes)]

[250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34]


In [21]:
#initializing the datetime variable which will be used for selecting values from the table
tec_maps = []
for j in range(10, 40, 5):
    date = datetime.datetime.strptime( "20150101"+ ":" + "00"+str(j), "%Y%m%d:%H%M" )
    #selecting those rows where the datetime gives the time with the longitudes are in the given range and latitude = 11
    check_tec_map = medFiltTECDF[(medFiltTECDF["Mlat"]==11.00) & (medFiltTECDF["Mlon"].isin(longitudes)) & (medFiltTECDF["date"]==date)]
    if(len(check_tec_map) >0):
        #populating the TEC maps in a matrix format. The rows corresponds to latitudes and columns corresponds to longitudes
        #Number of latitdues = (89-11) + 1 = 79, number of longitudes = (34-0)/2 + (360-250)/2 + 1 = 73. (number of longitudes is divided by 2 as the degree of longitudes is 2)
        #values in the TEC maps are stored from (250 to 360) then (0 to 34)
        tec_map = np.zeros((79,73,1))
        longi = list(range(0, 35, 2))
        for i in range(0, 79):
            selected = medFiltTECDF[(medFiltTECDF["Mlat"]==i+11) & (medFiltTECDF["Mlon"].isin(longitudes)) & (medFiltTECDF["date"]==date)]
            tec = []
            for index, row in selected.iterrows():
                if(int(row["Mlon"]) in longi):
                    j = int(row["Mlon"])/2 + 55
                else:
                    j = ((int(row["Mlon"]) - 250)/2)
                tec_map[i][j] = [float(row["med_tec"])]
        print tec_map.shape
        tec_maps.append(tec_map)
        
tec_maps = np.asarray(tec_maps)
print tec_maps.shape

(79, 73, 1)
(79, 73, 1)
(79, 73, 1)
(79, 73, 1)
(79, 73, 1)
(5, 79, 73, 1)


In [22]:
#saving the tec_maps in a pickle file
f = open('tec_maps.pkl','w')
pickle.dump(tec_maps, f)
f.close()